In [1]:
import numpy as np
from cnn2 import *

In [2]:
batch_size = 64
conv1 = Conv2D([batch_size,28,28,1],12,5,1)
relu1 = Relu(conv1.output_shape)
pool1 = MaxPooling(relu1.output_shape)
conv2 = Conv2D(pool1.output_shape,24,3,1)
relu2 = Relu(conv2.output_shape)
pool2 = MaxPooling(relu2.output_shape)
fc = FullyConnect(pool2.output_shape,10)
sf = Softmax(fc.output_shape)